# **Table des matières**

[**Imports et fonctions**](#Imports-et-fonctions)

[**Exploration et préparation des données**](#Exploration-et-préparation)
   - [Cinq datasets](#Cinq-datasets)
   - [population](#population)
   - [sous_nutrition](#sous_nutrition)
   - [aliments (vegetaux + animaux)](#aliments-(vegetaux-+-animaux))
   - [cereales](#cereales)
   - [Insights](#Insights)

[**Questions 1 à 14**](#Questions-1-à-14) 
   - [Question 1 : Nombre d'humains sur la planète](#Q1-:-Nombre-d'humains-sur-la-planète)
   - [Question 2 : Redondances](#Q2-:-Redondances)
   - [Question 3 : Dispo. alimentaire (calories, protéines)](#Q3-:-Dispo.-alimentaire-(calories,-protéines))
   - [Question 4 : Ratio énergie/poids](#Q4-:-Ratio-énergie/poids)
   - [Question 5 : Aliments les plus caloriques et protéiques](#Q5-:-Aliments-les-plus-caloriques-et-protéiques)
   - [Question 6 : Dispo. intérieure mondiale des végétaux](#Q6-:-Dispo.-intérieure-mondiale-des-végétaux-(calories))
   - [Question 7 : Potentiel alimentaire des végétaux](#Q7-:-Potentiel-alimentaire-des-végétaux)
   - [Question 8 : Potentiel alimentaire des végétaux (destinés aux animaux pertes)](#Q8-:-Potentiel-alimentaire-des-végétaux-(destinés-aux-animaux-+-pertes))
   - [Question 9 : Potentiel alimentaire de la dispo. mondiale](#Q9-:-Potentiel-alimentaire-de-la-dispo.-mondiale)
   - [Question 10 : Pourcentage de la sous-nutrition mondiale](#Q10-:-Pourcentage-de-la-sous-nutrition-mondiale)
   - [Question 11 : Céréales](#Q11-:-Céréales)
   - [Question 12 : Sous-nutrition](#Q12-:-Sous-nutrition)
   - [Question 13 : USA](#Q13-:-USA)
   - [Question 14 : Thaïlande](#Q14-:-Thaïlande)

[**Questions 15 à 20**](#Questions-15-à-20)
   - [Question 15 : Table population](#Q15-:-Table-population)
   - [Question 16 : Table dispo_alim](#Q16-:-Table-dispo_alim)
   - [Question 17 : Table equilibre_prod](#Q17-:-Table-equilibre_prod)
   - [Question 18 : Table sous_nutrition](#Q18-:-Table-sous_nutrition)
   - [Question 19 : Requêtes SQL](#Q19-:-Requêtes-SQL)
   - [Question 20 : ](#Q20-:-Table-population)

[**Analyses supplémentaires**](#Analyses-supplémentaires)
   - [Nombre de personnes sous-alimentées](#Nombre-de-personnes-sous-alimentées)
   - [Décès dus à la faim](#Décès-dus-à-la-faim)
   - [Évolutions possibles pour 2050](#Évolutions-possibles-pour-2050)
   - [Causes de la sous-alimentation](#Causes-de-la-sous-alimentation)      

# **Imports et fonctions**

In [1]:
# Import des librairies
import numpy as np
import pandas as pd
import sqlite3

In [2]:
# Import des fichiers et création des dataframes
url = 'sources/fr_'
population =       pd.read_csv(url + 'population' + '.csv')
sous_nutrition =   pd.read_csv(url + 'sousalimentation' + '.csv')    
animaux =          pd.read_csv(url + 'animaux' + '.csv')
cereales =         pd.read_csv(url + 'cereales' + '.csv')    
vegetaux =         pd.read_csv(url + 'vegetaux' + '.csv')

In [3]:
# Liste des dataframes
listDf = [population, sous_nutrition, animaux, cereales, vegetaux]
listDfStr = ['population', 'sous_nutrition', 'animaux', 'cereales', 'vegetaux']

In [4]:
# Fonction d'exploration des datasets
def exploration(df):
    
    # Noms des colonnes, null-count, types d'objets 
    print(df.info())
    
    print('\n')
    for i in range(0, len(df.columns)):
        
        # Nom de la colonne
        print(df.columns[i], ':')
        
        # Nombre de Nan 
        print(sum(df.iloc[:, i].isna()), 'Nan')
        
        # Nombre de zéros
        print(df.shape[0] - np.count_nonzero(df.iloc[:, i]), 'zéros')
        
        # Nombre de valeurs uniques 
        print(len(df.iloc[:, i].unique()), 'valeurs uniques :')
        print(df.iloc[:, i].unique())
        
        # .describe()
        print(df.iloc[:, i].describe())
        
        print('\n')

# **Exploration et préparation**

## Cinq datasets
- Les datasets contiennent les mêmes nombres et intitulés de colonnes, à l'exception de sous_nutrition, qui compte en plus la colonne 'Note'

In [5]:
# Renommer les différentes colonnes des datasets
for i in range(0, len(listDf)):
    listDf[i].rename(columns={
        'Code zone' : 'code_pays',
        'Zone' : 'pays',
        'Année' : 'annee',
        'Produit' : 'produit',
        'Code Produit' : 'code_produit'
    }, inplace=True)    

In [6]:
# Exploration des 5 datasets
for i in range(0, len(listDf)):
    print(
        listDfStr[i], ':',
        
        # Dimensions
        listDf[i].shape[1], 'colonnes, ',
        listDf[i].shape[0], 'lignes \n \n',
        
        # Noms des colonnes 
        listDf[i].columns, '\n \n'
    )

population : 14 colonnes,  175 lignes 
 
 Index(['Code Domaine', 'Domaine', 'code_pays', 'pays', 'Code Élément',
       'Élément', 'code_produit', 'produit', 'Code année', 'annee', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole'],
      dtype='object') 
 

sous_nutrition : 15 colonnes,  1020 lignes 
 
 Index(['Code Domaine', 'Domaine', 'code_pays', 'pays', 'Code Élément',
       'Élément', 'code_produit', 'produit', 'Code année', 'annee', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole', 'Note'],
      dtype='object') 
 

animaux : 14 colonnes,  37166 lignes 
 
 Index(['Code Domaine', 'Domaine', 'code_pays', 'pays', 'Code Élément',
       'Élément', 'code_produit', 'produit', 'Code année', 'annee', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole'],
      dtype='object') 
 

cereales : 14 colonnes,  891 lignes 
 
 Index(['Code Domaine', 'Domaine', 'code_pays', 'pays', 'Code Élément',
       'Élément', 'code_produit', 'produit', 'Code année', 'anne

## population
- A l'exception de 'Code zone', 'Zone', 'Valeur', 'Symbole' et 'Description du Symbole', toutes les colonnes contiennent une seule valeur
- Ces colonnes ne sont pas des clés, elles sont inutiles à l'exception de 'Année' (voir table [sous_nutrition](#sous_nutrition))
- Les colonnes 'Symbole' et 'Description du Symbole' ne contiennent que 2 valeurs indiquant la présence d'un agrégat (voir [question 1](#Question-1))
- Les colonnes 'Zone' et 'Code zone' contiennent les pays et leur code (175 au total, soit 19 de mois que le [nombre d'états membres de la FAO](http://www.fao.org/about/who-we-are/fr/#:~:text=La%20FAO%20est%20une%20organisation,membre%2C%20l'Union%20europ%C3%A9enne.))

In [7]:
exploration(population)    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Code Domaine            175 non-null    object
 1   Domaine                 175 non-null    object
 2   code_pays               175 non-null    int64 
 3   pays                    175 non-null    object
 4   Code Élément            175 non-null    int64 
 5   Élément                 175 non-null    object
 6   code_produit            175 non-null    int64 
 7   produit                 175 non-null    object
 8   Code année              175 non-null    int64 
 9   annee                   175 non-null    int64 
 10  Unité                   175 non-null    object
 11  Valeur                  175 non-null    int64 
 12  Symbole                 1 non-null      object
 13  Description du Symbole  175 non-null    object
dtypes: int64(6), object(8)
memory usage: 19.3+ KB
None


Code 

In [8]:
# Renommer la colonne 'Valeur'
population.rename(columns={'Valeur' : 'population'}, inplace=True)

# Projection des colonnes contenant plus d'une valeur (excepté 'annee')
# Conserver temporairement 'Description du Symbole', en prévision de la question 1
population = population[['code_pays','pays','population','annee','Description du Symbole']]

# Conversion de la population en habitants
population['population'] = population['population'] * 1000

# Affichage
population.sample(3)

<ipython-input-8-281ff81a35e9>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population['population'] = population['population'] * 1000


,code_pays,pays,population,annee,Description du Symbole
27,233,Burkina Faso,16935000,2013,Donnée officielle
127,170,Pérou,30376000,2013,Donnée officielle
42,107,Côte d'Ivoire,20316000,2013,Donnée officielle


## sous_nutrition
- Seules les colonnes 'Code zone', 'Zone', 'Code année', 'Année', 'Valeur', 'Symbole et Description du Symbole' contiennent plus d'une valeur
- Contrairement à population, la colonne 'Unité' référence en millions la colonne 'Valeur' (mais la colonne valeur contient 40% de nulls)
- La colonne 'Valeur' contient des valeurs non numériques ('<0.1')
- Cette même colonne 'Valeur' n'est pas de type int, et contient des strings
- La colonne 'Note' est vide (seule colonne non commune aux autres tables)
- Contrairement à population, animaux et vegetaux, on compte ici plus de pays (204) que le FAO ne compte d'états membres
- Contrairement aux autres tables, la colonne année va de 2012-2014 à 2016-2018

In [9]:
exploration(sous_nutrition)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            1020 non-null   object 
 1   Domaine                 1020 non-null   object 
 2   code_pays               1020 non-null   int64  
 3   pays                    1020 non-null   object 
 4   Code Élément            1020 non-null   int64  
 5   Élément                 1020 non-null   object 
 6   code_produit            1020 non-null   int64  
 7   produit                 1020 non-null   object 
 8   Code année              1020 non-null   int64  
 9   annee                   1020 non-null   object 
 10  Unité                   1020 non-null   object 
 11  Valeur                  605 non-null    object 
 12  Symbole                 1020 non-null   object 
 13  Description du Symbole  1020 non-null   object 
 14  Note                    0 non-null      

In [10]:
# Renommer la colonne 'Valeur'
sous_nutrition.rename(columns={
    'Valeur' : 'nb_personnes'
}, inplace=True)

# Projection
sous_nutrition = sous_nutrition[['code_pays','pays','nb_personnes','annee']]

# Remplacer les '<0.1' (par 0)
sous_nutrition['nb_personnes'].replace('<0.1', '0', inplace=True)

# La colonne nb_personnes ne contient que des strings
print('Type des valeurs de la colonne \'nb_personnes\' :', type(sous_nutrition['nb_personnes'].loc[5]), '\n')

# Convertir la colonne nb_personnes en float
sous_nutrition['nb_personnes'] = sous_nutrition['nb_personnes'].astype(float)

# Conversion de nb_personnes en habitants
sous_nutrition['nb_personnes'] = sous_nutrition['nb_personnes'] * 1000000

# Affichage
print(sous_nutrition.sample(3))

Type des valeurs de la colonne 'nb_personnes' : <class 'str'> 

     code_pays      pays  nb_personnes      annee
113         17  Bermudes           NaN  2015-2017
652        160     Nioué           NaN  2014-2016
621        147   Namibie      700000.0  2013-2015


c:\users\guifo\appdata\local\programs\python\python38\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-10-38c88a28aea6>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sous_nutrition['nb_personnes'] = sous_nutrition['nb_personnes'].astype(float)
<ipython-input-10-38c88a28aea6>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

## aliments (vegetaux + animaux)
- Ces 2 tables comportent les mêmes structures et valeurs, à l'exception des colonnes référençant les produits et valeurs

In [11]:
exploration(animaux)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37166 entries, 0 to 37165
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            37166 non-null  object 
 1   Domaine                 37166 non-null  object 
 2   code_pays               37166 non-null  int64  
 3   pays                    37166 non-null  object 
 4   Code Élément            37166 non-null  int64  
 5   Élément                 37166 non-null  object 
 6   code_produit            37166 non-null  int64  
 7   produit                 37166 non-null  object 
 8   Code année              37166 non-null  int64  
 9   annee                   37166 non-null  int64  
 10  Unité                   37166 non-null  object 
 11  Valeur                  37166 non-null  float64
 12  Symbole                 37166 non-null  object 
 13  Description du Symbole  37166 non-null  object 
dtypes: float64(1), int64(5), object(8)
mem

In [12]:
exploration(vegetaux)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104871 entries, 0 to 104870
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Code Domaine            104871 non-null  object 
 1   Domaine                 104871 non-null  object 
 2   code_pays               104871 non-null  int64  
 3   pays                    104871 non-null  object 
 4   Code Élément            104871 non-null  int64  
 5   Élément                 104871 non-null  object 
 6   code_produit            104871 non-null  int64  
 7   produit                 104871 non-null  object 
 8   Code année              104871 non-null  int64  
 9   annee                   104871 non-null  int64  
 10  Unité                   104871 non-null  object 
 11  Valeur                  104871 non-null  float64
 12  Symbole                 104871 non-null  object 
 13  Description du Symbole  104871 non-null  object 
dtypes: float64(1), int64

0 zéros
1 valeurs uniques :
[2013]
count    104871.0
mean       2013.0
std           0.0
min        2013.0
25%        2013.0
50%        2013.0
75%        2013.0
max        2013.0
Name: annee, dtype: float64


Unité :
0 Nan
0 zéros
4 valeurs uniques :
['Milliers de tonnes' 'kg' 'Kcal/personne/jour' 'g/personne/jour']
count                 104871
unique                     4
top       Milliers de tonnes
freq                   67783
Name: Unité, dtype: object


Valeur :
0 Nan
32006 zéros
6249 valeurs uniques :
[5169.   1173.   -350.   ... 3880.     28.7    20.63]
count    104871.000000
mean        321.271016
std        6395.039684
min      -39863.000000
25%           0.000000
50%           1.000000
75%          17.000000
max      739267.000000
Name: Valeur, dtype: float64


Symbole :
0 Nan
0 zéros
3 valeurs uniques :
['S' 'Fc' 'A']
count     104871
unique         3
top            S
freq       67182
Name: Symbole, dtype: object


Description du Symbole :
0 Nan
0 zéros
3 valeurs uniques :
[

In [13]:
# Valeurs inférieures à 0 ??? 
print('Animaux : \n', animaux['Valeur'].describe(), '\n')
print(vegetaux['Valeur'].describe())

Animaux : 
 count     37166.000000
mean        148.291647
std        1933.728473
min       -1368.000000
25%           0.000000
50%           1.000000
75%          11.000000
max      135600.000000
Name: Valeur, dtype: float64 

count    104871.000000
mean        321.271016
std        6395.039684
min      -39863.000000
25%           0.000000
50%           1.000000
75%          17.000000
max      739267.000000
Name: Valeur, dtype: float64


In [14]:
# Ajout d'une colonne 'Origine' dans animaux et vegetaux
animaux['origine'] = 'Animale'
vegetaux['origine'] = 'Végétale'

# Union de Végétaux et Animaux
aliments = pd.concat([vegetaux, animaux])

# Projection
aliments = aliments[[
    'pays',
    'code_pays',
    'produit',
    'code_produit',
    'origine',
    'Élément',
    'Valeur',
    'annee'
]]

# Table pivot sur aliments
aliments = aliments.pivot_table('Valeur', index=['code_pays','pays','produit','code_produit','annee','origine'], columns='Élément')

# Reset index
aliments.reset_index(inplace=True)

# Renommer les colonnes
aliments.rename(columns={
    'Aliments pour animaux' : 'alim_ani',
    'Autres utilisations (non alimentaire)' : 'autres_utilisations',
    'Semences' : 'semences',
    'Pertes' : 'pertes',
    'Nourriture' : 'nourriture',
    'Traitement' : 'transfo', # ?
    'Disponibilité intérieure' : 'dispo_int',
    'Disponibilité alimentaire (Kcal/personne/jour)' : 'dispo_alim_kcal_p_j',
    'Disponibilité de protéines en quantité (g/personne/jour)' : 'dispo_prot',
    'Disponibilité de matière grasse en quantité (g/personne/jour)' : 'dispo_mat_gr'
}, inplace=True)

# Joindre population et aliments
aliments = pd.merge(aliments, population[['pays', 'population']], on="pays")

# Remplacer les Nan
aliments.replace(np.nan, 0, inplace=True)

# Affichage
print('Résultat : ', aliments.shape[1], 'colonnes, ', aliments.shape[0], 'lignes')
aliments.sample(3)

Résultat :  22 colonnes,  15702 lignes


,code_pays,pays,produit,code_produit,annee,origine,alim_ani,autres_utilisations,dispo_alim_kcal_p_j,Disponibilité alimentaire en quantité (kg/personne/an),...,dispo_int,Exportations - Quantité,Importations - Quantité,nourriture,pertes,Production,semences,transfo,Variation de stock,population
11667,185,Fédération de Russie,Seigle,2515,2013,Végétale,1548.0,0.0,56.0,7.63,...,3355.0,55.0,51.0,1090.0,105.0,3360.0,413.0,200.0,0.0,142834000
13957,225,Émirats arabes unis,Pois,2547,2013,Végétale,0.0,3.0,40.0,4.22,...,42.0,9.0,51.0,39.0,0.0,0.0,0.0,0.0,0.0,9346000
4458,73,Géorgie,Miel,2745,2013,Végétale,0.0,0.0,7.0,0.90,...,4.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,4341000


In [15]:
exploration(aliments)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15702 entries, 0 to 15701
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   code_pays                                               15702 non-null  int64  
 1   pays                                                    15702 non-null  object 
 2   produit                                                 15702 non-null  object 
 3   code_produit                                            15702 non-null  int64  
 4   annee                                                   15702 non-null  int64  
 5   origine                                                 15702 non-null  object 
 6   alim_ani                                                15702 non-null  float64
 7   autres_utilisations                                     15702 non-null  float64
 8   dispo_alim_kcal_p_j                 

3608 zéros
2928 valeurs uniques :
[ 4.59  0.46  0.   ... 40.75 77.45 38.6 ]
count    15702.000000
mean         7.839705
std         23.548134
min         -1.930000
25%          0.010000
50%          0.520000
75%          4.267500
max        430.760000
Name: Disponibilité alimentaire en quantité (kg/personne/an), dtype: float64


dispo_mat_gr :
0 Nan
6577 zéros
1129 valeurs uniques :
[3.900e-01 0.000e+00 1.000e-02 ... 4.070e+00 5.170e+00 3.527e+01]
count    15702.000000
mean         0.969869
std          3.250011
min         -0.030000
25%          0.000000
50%          0.020000
75%          0.310000
max         60.760000
Name: dispo_mat_gr, dtype: float64


dispo_prot :
0 Nan
6725 zéros
1038 valeurs uniques :
[2.250e+00 1.000e-02 0.000e+00 ... 3.120e+00 1.132e+01 5.850e+00]
count    15702.000000
mean         0.907157
std          3.141913
min         -0.370000
25%          0.000000
50%          0.020000
75%          0.350000
max         54.970000
Name: dispo_prot, dtype: float64


dispo

6537 zéros
903 valeurs uniques :
[ 2.0000e+00  1.0000e+00  0.0000e+00  8.0000e+00  5.0000e+00  3.0000e+00
  3.6100e+02  1.4000e+01  4.0000e+00  7.0000e+00  6.0000e+00  2.9000e+01
  3.0000e+01  8.2000e+01  1.5000e+01  9.0000e+00  1.1000e+01  1.0000e+01
  8.5000e+01  1.9000e+01  3.2000e+01  4.0000e+01  1.1730e+03  1.8000e+01
  1.6000e+01  3.6000e+02  1.1900e+02  2.2000e+01  2.4800e+02  1.2400e+02
  2.6000e+01  1.3900e+02  2.7100e+02  1.0700e+02  4.8000e+01  2.0000e+01
  2.8000e+01  9.6000e+01  1.7000e+01  2.3000e+01  6.8000e+01  1.2000e+01
  2.7000e+01  2.5000e+01  6.5000e+01  5.3000e+01  2.7400e+02  6.3430e+03
  1.2900e+02  2.4000e+01  4.1000e+01  5.5000e+01  6.3000e+01  7.9000e+01
  1.2000e+02  6.2300e+02  6.2000e+01  2.5810e+03  8.0000e+01  1.9300e+02
  3.2510e+03  3.6000e+01  5.2000e+01  5.5300e+02  1.1600e+02  1.6000e+02
  3.4000e+01  1.1800e+02  1.8360e+03  1.3000e+01  1.7600e+02  1.6400e+02
  9.7200e+02  3.3000e+01  2.2400e+02  7.2000e+01  9.0000e+01  8.5200e+02
  9.9000e+01  9.40

## cereales
Cette table ne contient que des redondances de la table vegetaux, elle ne servira qu'à catégoriser certains de ces produits végétaux.

In [16]:
exploration(cereales)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Code Domaine            891 non-null    object
 1   Domaine                 891 non-null    object
 2   code_pays               891 non-null    int64 
 3   pays                    891 non-null    object
 4   Code Élément            891 non-null    int64 
 5   Élément                 891 non-null    object
 6   code_produit            891 non-null    int64 
 7   produit                 891 non-null    object
 8   Code année              891 non-null    int64 
 9   annee                   891 non-null    int64 
 10  Unité                   891 non-null    object
 11  Valeur                  891 non-null    int64 
 12  Symbole                 891 non-null    object
 13  Description du Symbole  891 non-null    object
dtypes: int64(6), object(8)
memory usage: 97.6+ KB
None


Code 

['S']
count     891
unique      1
top         S
freq      891
Name: Symbole, dtype: object


Description du Symbole :
0 Nan
0 zéros
1 valeurs uniques :
['Données standardisées']
count                       891
unique                        1
top       Données standardisées
freq                        891
Name: Description du Symbole, dtype: object




## *Insights*
- L'ensemble des tables couvre uniquement l'année 2013, excepté sous_nutrition, qui contient des données de 2012 à 2015 : on a donc gardé la colonne 'annee' pour population et sous_nutrition
- Présence de Nan et de zéros :
    - **population** : aucun
    - **sous_nutrition** : 40% de Nan dans la colonne 'nb_habitants'
    - **aliments** (avant pivot) : 32% de zéros dans la colonne 'Valeur'
    - **aliments** (après pivot) : de 25% à 90% de zéros dans toutes les colonnes (excepté 'population')

# **Questions 1 à 14**

## <a id="Q1">Q1 : Nombre d'humains sur la planète</a>
Calculez le nombre total d’humains sur la planète. Critiquez votre résultat. En cas d’anomalie, analysez et effectuer les corrections nécessaires. Donnez le résultat de votre calcul pour l'année 2013.

### Surestimation

In [17]:
population_monde = population['population'].sum()
print("Population mondiale : ⚠️", round((population_monde) / 10**9, 2), "milliards d'habitants")

Population mondiale : ⚠️ 8.41 milliards d'habitants


### La Chine comptabilisée 2 fois

Le résultat (8.41 milliards) excède la population mondiale estimée pour 2013 (7.21 milliards selon [Wikipedia](https://en.wikipedia.org/wiki/World_population)). 
En cause : les données de la Chine, comptabilisées 2 fois (données d'origine et agrégat).

In [18]:
population[population['pays'].str.contains('Chine')]

,code_pays,pays,population,annee,Description du Symbole
33,351,Chine,1416667000,2013,"Agrégat, peut inclure des données officielles,..."
34,96,Chine - RAS de Hong-Kong,7204000,2013,Donnée officielle
35,128,Chine - RAS de Macao,566000,2013,Donnée officielle
36,41,"Chine, continentale",1385567000,2013,Donnée officielle
37,214,"Chine, Taiwan Province de",23330000,2013,Donnée officielle


In [19]:
somme_agregat = population[population['code_pays'].isin([96,128,41,214])]['population'].sum()
print('Somme des codes 96, 128, 41 et 214 :', somme_agregat, 'millions (identique au code pays 351)')

Somme des codes 96, 128, 41 et 214 : 1416667000 millions (identique au code pays 351)


In [20]:
# Fonction de suppression de l'agrégat (code pays 351)
def suppression_agregat(df):
    df.drop(df[df['code_pays']==351].index, inplace=True)
    
# Suppression de l'agrégat dans les dataframes
suppression_agregat(population)
suppression_agregat(sous_nutrition)
suppression_agregat(aliments)

# Drop de la colonne Description dans population, maintenant inutile
population.drop(columns=['Description du Symbole'], inplace=True)

print(population[population['pays'].str.contains('Chine')])

    code_pays                       pays  population  annee
34         96   Chine - RAS de Hong-Kong     7204000   2013
35        128       Chine - RAS de Macao      566000   2013
36         41        Chine, continentale  1385567000   2013
37        214  Chine, Taiwan Province de    23330000   2013


c:\users\guifo\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\users\guifo\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\users\guifo\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

### 7 milliards après correction
La nouvelle somme des populations avoisine alors les 7 milliards, chiffre plus proche de la réalité en 2013 (mais légèrement inférieur aux 7,15 milliards estimés par l'ONU à cette date).

In [21]:
# Population mondiale corrigée
population_monde = population['population'].sum()
print("Population mondiale (corrigée) : 👌", round((population_monde) / 10**9, 2), "milliards d'habitants")

Population mondiale (corrigée) : 👌 7.0 milliards d'habitants


## Q2 : Redondances
Identifiez ces redondances, en donnant votre réponse sous forme de formule mathématique (pas besoin de coder ici :soleil: ). C'est une équation à 3 termes de type (a_1 + a2 + [...] = b_1 + b_2 + [...] = c_1 + c_2 + [...]) ) faisant intervenir chacune des 11 quantités données ci dessus. Illustrez cette équation avec l'exemple du blé en France.

In [22]:
# Formule
# Disponibilité intérieure = Production + Importations + Variation - Exportations = Nourriture + Aliments animaux + Semences + Pertes + Traitement + Autres utilisations

# Exemple du blé en France
bf = aliments[(aliments['pays']=='France') & (aliments['produit']=='Blé')].reset_index()

print(
    bf.loc[0, 'dispo_int'],
    '=',
    bf.loc[0, 'Production']
    + bf.loc[0, 'Importations - Quantité']
    + bf.loc[0, 'Variation de stock']
    - bf.loc[0, 'Exportations - Quantité'],
    '=',
    bf.loc[0, 'nourriture']
    + bf.loc[0, 'alim_ani']
    + bf.loc[0, 'semences']
    + bf.loc[0, 'pertes']
    + bf.loc[0, 'transfo']
    + bf.loc[0, 'autres_utilisations']
)      

20298.0 = 20298.0 = 20298.0


## Q3 : Dispo. alimentaire (calories, protéines)
Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines. Vous ferez cela à partir de ces informations : Population de chaque pays; Disponibilité alimentaire donnée pour chaque produit et pour chaque pays en kcal/personne/jour, Disponibilité alimentaire en protéines donnée pour chaque produit et pour chaque pays en g/personne/jour.

In [23]:
# Création des colonnes Disponibilité en kcal/an et kgprot/an
aliments['dispo_kcal_an'] = aliments['dispo_alim_kcal_p_j'] * aliments['population'] * 365
aliments['dispo_kgprot_an'] = aliments['dispo_prot'] / 1000 * aliments['population'] * 365

print(len(aliments))
aliments.head(3)

15605


,code_pays,pays,produit,code_produit,annee,origine,alim_ani,autres_utilisations,dispo_alim_kcal_p_j,Disponibilité alimentaire en quantité (kg/personne/an),...,Importations - Quantité,nourriture,pertes,Production,semences,transfo,Variation de stock,population,dispo_kcal_an,dispo_kgprot_an
0,1,Arménie,Abats Comestible,2736,2013,Animale,0.0,3.0,14.0,4.59,...,2.0,14.0,0.0,15.0,0.0,0.0,0.0,2977000,1.521247e+10,2444861.25
1,1,Arménie,"Agrumes, Autres",2614,2013,Végétale,0.0,0.0,0.0,0.46,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2977000,0.000000e+00,10866.05
2,1,Arménie,"Alcool, non Comestible",2659,2013,Végétale,0.0,0.0,0.0,0.00,...,1.0,0.0,0.0,3.0,0.0,0.0,0.0,2977000,0.000000e+00,0.00


In [24]:
# Affichage par pays
print(aliments.pivot_table(index='pays', aggfunc='sum')[['dispo_kcal_an','dispo_kgprot_an']].head(3), '\n')

# Affichage par produit
print(aliments.pivot_table(index='produit', aggfunc='sum')[['dispo_kcal_an','dispo_kgprot_an']].head(3))

                dispo_kcal_an  dispo_kgprot_an
pays                                          
Afghanistan      2.327314e+13     6.496852e+08
Afrique du Sud   5.817498e+13     1.643732e+09
Albanie          3.692166e+12     1.289826e+08 

                        dispo_kcal_an  dispo_kgprot_an
produit                                               
Abats Comestible         1.723963e+13     2.799873e+09
Agrumes, Autres          2.473770e+12     5.205833e+07
Alcool, non Comestible   0.000000e+00     0.000000e+00


## Q4 : Ratio énergie/poids
A partir de ces dernières informations, et à partir du poids de la disponibilité alimentaire (pour chaque pays et chaque produit), calculez pour chaque produit le ratio "énergie/poids", que vous donnerez en kcal/kg. Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la valeur calorique d'un oeuf.

In [25]:
# Création des colonnes ratios kcal/kg et pourcentage de protéines
aliments['kcal/kg'] = aliments['dispo_kcal_an'].replace(0, np.nan) / (aliments['nourriture'].replace(0, np.nan) * 1000000)
aliments['% protéines'] = aliments['dispo_kgprot_an'].replace(0, np.nan) / (aliments['nourriture'].replace(0, np.nan) * 1000000) * 100

aliments.dropna(subset=['% protéines'])[aliments['produit']=='Oeufs'][['pays','produit','kcal/kg','% protéines']].sample(3)

<ipython-input-25-6ca98b1f9e72>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  aliments.dropna(subset=['% protéines'])[aliments['produit']=='Oeufs'][['pays','produit','kcal/kg','% protéines']].sample(3)


,pays,produit,kcal/kg,% protéines
9845,Macédoine du Nord,Oeufs,1419.793846,11.003402
11554,Rwanda,Oeufs,2149.302500,10.746512
15114,Zambie,Oeufs,1216.126771,10.502913


## Q5 : Aliments les plus caloriques et protéiques
Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids.
Citez 5 aliments parmi les 20 aliments les plus riches en protéines.

In [26]:
# 20 aliments les plus caloriques
print(aliments.pivot_table('kcal/kg', index='produit').sort_values(by='kcal/kg', ascending=False).head(20), '\n')

# 20 aliments les plus protéiques
print(aliments.pivot_table('% protéines', index='produit').sort_values(by='% protéines', ascending=False).head(20), '\n')

                              kcal/kg
produit                              
Huiles de Foie de Poisso  9982.932500
Huile de Sésame           9433.744729
Huile de Son de Riz       9361.549883
Huiles de Poissons        9003.715504
Huile d'Arachide          8935.765811
Huile d'Olive             8931.223447
Huile de Germe de Maïs    8871.762566
Huil Plantes Oleif Autr   8848.222893
Huile de Palmistes        8789.540277
Huile Graines de Coton    8750.320136
Huile de Palme            8719.419872
Huile de Tournesol        8693.245772
Huile de Soja             8635.808245
Huile de Colza&Moutarde   8627.372452
Huile de Coco             8565.739255
Graisses Animales Crue    7583.862063
Beurre, Ghee              7331.424291
Palmistes                 6496.635000
Sésame                    5796.120413
Arachides Decortiquees    5702.900100 

                          % protéines
produit                              
Graines Colza/Moutarde      28.536511
Soja                        28.533053
Arachides 

In [27]:
# Vérification avec les graines de colza/moutarde
aliments[aliments['produit']=='Graines Colza/Moutarde']['% protéines'].mean()

28.536511111111114

## Q6 : Dispo. intérieure mondiale des végétaux (calories)
Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.

In [28]:
# Colonnes Disponibilité intérieure en kcal
aliments['dispo_int_kcal'] = aliments['dispo_int'].replace(0, np.nan) * 1000000 * aliments['kcal/kg']

# Disponibilité intérieure mondiale des végétaux (kcal)
dispo_int_kcal_vegetaux = aliments[aliments['origine']=='Végétale']['dispo_int_kcal'].sum()

print(dispo_int_kcal_vegetaux, 'kcal')

1.2312091265143276e+16 kcal


## Q7 : Potentiel alimentaire des végétaux
Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [29]:
# Besoin calorique annuel / personne
besoin_kcal = 2300 * 365

# Humains qu'on pourrait nourrir avec la dispo. intérieure en végétaux
bouches_kcal_vegetaux = dispo_int_kcal_vegetaux / besoin_kcal

print('En calories :')
print(round(bouches_kcal_vegetaux / 10**9), 'milliards d\'humains')
print(round(bouches_kcal_vegetaux / population_monde * 100), '% de la population mondiale')

En calories :
15 milliards d'humains
210 % de la population mondiale


In [30]:
# Colonnes Disponibilité intérieure en kgprot
aliments['dispo_int_kgprot'] = aliments['dispo_int'].replace(0, np.nan) * 1000000 * aliments['% protéines'] / 100

# Disponibilité intérieure mondiale des végétaux (kgprot)
dispo_int_kgprot_vegetaux = aliments[aliments['origine']=='Végétale']['dispo_int_kgprot'].sum()

# Besoin protéique annuel / personne
besoin_kgprot = .9 * 62 * 365 /1000

# Humains qu'on pourrait nourrir avec la dispo. intérieure en végétaux
bouches_kgprot_vegetaux = dispo_int_kgprot_vegetaux / besoin_kgprot

print('En protéines :')
print(round(bouches_kgprot_vegetaux / 10**9), 'milliards d\'humains')
print(round(bouches_kgprot_vegetaux / population_monde * 100), '% de la population mondiale')

En protéines :
14 milliards d'humains
204 % de la population mondiale


## Q8 : Potentiel alimentaire des végétaux (destinés aux animaux + pertes)
Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux la nourriture végétale destinée aux animaux et les pertes de produits végétaux étaient utilisés pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

### En remplaçant les 0 par des Nan, puis dropna

In [31]:
vegetaux_nourriture = aliments[aliments['origine']=='Végétale']['nourriture'].replace(0, np.nan).dropna()
vegetaux_alim_ani = aliments[aliments['origine']=='Végétale']['alim_ani'].replace(0, np.nan).dropna()
vegetaux_pertes = aliments[aliments['origine']=='Végétale']['pertes'].replace(0, np.nan).dropna()

aliments['vegetaux_q8_kcal'] = (vegetaux_nourriture + vegetaux_alim_ani + vegetaux_pertes) * 1000000 * aliments['kcal/kg']
aliments['vegetaux_q8_kgprot'] = (vegetaux_nourriture + vegetaux_alim_ani + vegetaux_pertes) * 1000000 * aliments['% protéines'] / 100


# Somme des aliments pour animaux et pertes (pour les végétaux)
somme_q8_kcal = aliments['vegetaux_q8_kcal'].sum()
somme_q8_kgprot = aliments['vegetaux_q8_kgprot'].sum()

bouches_q8_kcal = somme_q8_kcal / besoin_kcal
bouches_q8_kgprot = somme_q8_kgprot / besoin_kgprot

print('En calories :')
print(round((bouches_q8_kcal / 10**9), 2), 'milliards d\'humains')
print(round(bouches_q8_kcal / population_monde * 100), '% de la population mondiale \n')

print('En protéines :')
print(round((bouches_q8_kgprot / 10**9), 2), 'milliards d\'humains')
print(round(bouches_q8_kgprot / population_monde * 100), '% de la population mondiale')

En calories :
7.11 milliards d'humains
102 % de la population mondiale 

En protéines :
7.68 milliards d'humains
110 % de la population mondiale


### En ne remplçant pas les 0 par des Nan

In [32]:
vegetaux_nourriture = aliments[aliments['origine']=='Végétale']['nourriture']
vegetaux_alim_ani = aliments[aliments['origine']=='Végétale']['alim_ani']
vegetaux_pertes = aliments[aliments['origine']=='Végétale']['pertes']

aliments['vegetaux_q8_kcal'] = (vegetaux_nourriture + vegetaux_alim_ani + vegetaux_pertes) * 1000000 * aliments['kcal/kg']
aliments['vegetaux_q8_kgprot'] = (vegetaux_nourriture + vegetaux_alim_ani + vegetaux_pertes) * 1000000 * aliments['% protéines'] / 100


# Somme des aliments pour animaux et pertes (pour les végétaux)
somme_q8_kcal = aliments['vegetaux_q8_kcal'].sum()
somme_q8_kgprot = aliments['vegetaux_q8_kgprot'].sum()

bouches_q8_kcal = somme_q8_kcal / besoin_kcal
bouches_q8_kgprot = somme_q8_kgprot / besoin_kgprot

print('En calories :')
print(round((bouches_q8_kcal / 10**9), 2), 'milliards d\'humains')
print(round(bouches_q8_kcal / population_monde * 100), '% de la population mondiale \n')

print('En protéines :')
print(round((bouches_q8_kgprot / 10**9), 2), 'milliards d\'humains')
print(round(bouches_q8_kgprot / population_monde * 100), '% de la population mondiale')

En calories :
10.93 milliards d'humains
156 % de la population mondiale 

En protéines :
10.0 milliards d'humains
143 % de la population mondiale


## Q9 : Potentiel alimentaire de la dispo. mondiale
Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [33]:
# Disponibilités alimentaires en kcal et kgprot
aliments['dispo_kcal'] = aliments['nourriture'] * 1000000 * aliments['kcal/kg']
aliments['dispo_kgprot'] = aliments['nourriture'] * 1000000 * aliments['% protéines'] / 100

# Sommes de la disponibilité alimentaire mondiale
dispo_mondiale_kcal = aliments['dispo_kcal'].sum()
dispo_mondiale_kgprot = aliments['dispo_kgprot'].sum()

# Humains qu'on pourrait nourrir avec cette disponibilité
bouches_dispo_mondiale_kcal = dispo_mondiale_kcal / besoin_kcal
bouches_dispo_mondiale_kgprot = dispo_mondiale_kgprot / besoin_kgprot

print('En calories :')
print(round((bouches_dispo_mondiale_kcal / 10**9), 2), 'milliards d\'humains')
print(round(bouches_dispo_mondiale_kcal / population_monde * 100), '% de la population mondiale \n')

print('En protéines :')
print(round((bouches_dispo_mondiale_kgprot / 10**9), 2), 'milliards d\'humains')
print(round(bouches_dispo_mondiale_kgprot / population_monde * 100), '% de la population mondiale')

En calories :
8.77 milliards d'humains
125 % de la population mondiale 

En protéines :
10.17 milliards d'humains
145 % de la population mondiale


## Q10 : Pourcentage de la sous-nutrition mondiale
A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [34]:
# Restriction à l'année 2013
sous_nutrition = sous_nutrition[sous_nutrition['annee']=='2012-2014']
sous_nutrition['annee'] = '2013'

sous_nutrition.sample(3)

,code_pays,pays,nb_personnes,annee
665,156,Nouvelle-Zélande,NaN,2013
335,74,Gabon,200000.0,2013
145,27,Bulgarie,400000.0,2013


In [35]:
# Personnes sous-alimentées dans le monde
sousnutrition_monde = sous_nutrition['nb_personnes'].sum()
ratio_sousnutrition = sousnutrition_monde / population_monde

# Pourcentage de la population mondiale sous-alimentée
print('Sous-nutrition :', round(ratio_sousnutrition * 100), '% de la population mondiale')

Sous-nutrition : 11 % de la population mondiale


## Q11 : Céréales
Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO.
En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [36]:
# Liste des céréales
liste_cereales = cereales['produit'].drop_duplicates()
liste_cereales

0                  Blé
1     Riz (Eq Blanchi)
2                 Orge
3                 Maïs
4               Millet
9               Seigle
10              Avoine
12              Sorgho
13    Céréales, Autres
Name: produit, dtype: object

In [37]:
# Céréales destinées à l'alimentation humaine et animale
aliments_cereales = aliments[aliments['produit'].isin(liste_cereales)]

# Pourcentage de céréales destinées à l'alimentation animale
cereales_alim_ani = aliments_cereales['alim_ani'].sum() / (aliments_cereales['nourriture'].sum() + aliments_cereales['alim_ani'].sum()) * 100

print('Proportion des céréales destinée à l\'alimentation animale :', round(cereales_alim_ani), '%')

Proportion des céréales destinée à l'alimentation animale : 46 %


## Q12 : Sous-nutrition
Sélectionnez parmi les données des bilans alimentaires les informations relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition.

Repérez les 15 produits les plus exportés par ce groupe de pays.

Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné)

Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. Ensuite, calculez pour chaque produit les 2 quantités suivantes :

le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.
le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

Question 12 : Donnez les 3 produits qui on t la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)


In [38]:
# Jointure de sous_nutrition et population
sous_nutrition = pd.merge(sous_nutrition, population[['pays','population']], on='pays')

# Colonne pourcentage de sous-nutrition pour chaque pays
sous_nutrition['% sous_nutrition'] = sous_nutrition['nb_personnes'] / sous_nutrition['population'] * 100

sous_nutrition.head(3)

,code_pays,pays,nb_personnes,annee,population,% sous_nutrition
0,2,Afghanistan,7900000.0,2013,30552000,25.857554
1,202,Afrique du Sud,2600000.0,2013,52776000,4.926482
2,3,Albanie,200000.0,2013,3173000,6.303183


In [39]:
# Liste des pays en sous-nutrition (+ de 5% de sous-nutrition)
liste_pays_sousnutrition = sous_nutrition[sous_nutrition['% sous_nutrition'] > 5].sort_values(by=['% sous_nutrition'], ascending=False)['pays']

print(
    len(liste_pays_sousnutrition), 'pays sous-alimentés (',
    round(len(liste_pays_sousnutrition) / len(sous_nutrition) * 100), '% de la totalité ) \n \n',
    liste_pays_sousnutrition.head(10)
)

83 pays sous-alimentés ( 48 % de la totalité ) 
 
 70                                          Haïti
172                                        Zambie
173                                      Zimbabwe
131                     République centrafricaine
136    République populaire démocratique de Corée
39                                          Congo
157                                         Tchad
5                                          Angola
93                                        Libéria
97                                     Madagascar
Name: pays, dtype: object


In [40]:
# Les 15 produits les plus exportés par les pays en sous-nutrition
pays_sousnutrition_exportes = aliments[aliments['pays'].isin(liste_pays_sousnutrition)].pivot_table('Exportations - Quantité', index='produit').sort_values(by='Exportations - Quantité', ascending=False).head(15).reset_index()['produit']
pays_sousnutrition_exportes

0                  Manioc
1        Riz (Eq Blanchi)
2          Huile de Palme
3                 Bananes
4           Sucre Eq Brut
5                     Blé
6         Légumes, Autres
7                    Maïs
8     Poissons Pelagiques
9          Fruits, Autres
10     Lait - Excl Beurre
11                   Soja
12                 Ananas
13                Tomates
14                   Café
Name: produit, dtype: object

In [41]:
# Les 200 plus grandes importations de ces produits
imports_pays_sousnutrition_exportes = aliments[aliments['produit'].isin(pays_sousnutrition_exportes)].sort_values(by='Importations - Quantité', ascending=False).head(200)
imports_pays_sousnutrition_exportes[['pays','produit','Importations - Quantité']]

,pays,produit,Importations - Quantité
2567,"Chine, continentale",Soja,63381.0
2537,"Chine, continentale",Manioc,29046.0
7015,Japon,Maïs,14403.0
3686,Égypte,Blé,10331.0
7539,République de Corée,Maïs,8755.0
...,...,...,...
5685,Chine - RAS de Hong-Kong,"Fruits, Autres",1012.0
10278,Nigéria,Huile de Palme,999.0
3755,Égypte,Sucre Eq Brut,998.0
745,Autriche,Maïs,996.0


In [42]:
# Table pivot
quinze_produits_importes = imports_pays_sousnutrition_exportes.pivot_table(index='produit')

# Ratio autres/dispo.intérieure
quinze_produits_importes['autres_utilisations/dispo_int'] = quinze_produits_importes['autres_utilisations'] / quinze_produits_importes['dispo_int'] * 100

# Ratio nourriture animale / quantité destinée à la nourriture (animale + humaine)
quinze_produits_importes['alim_ani/nourriture_huma_ani'] = quinze_produits_importes['alim_ani'] / (quinze_produits_importes['alim_ani'] + quinze_produits_importes['nourriture']) * 100

quinze_produits_importes

,% protéines,Disponibilité alimentaire en quantité (kg/personne/an),Exportations - Quantité,Importations - Quantité,Production,Variation de stock,alim_ani,annee,autres_utilisations,code_pays,...,kcal/kg,nourriture,pertes,population,semences,transfo,vegetaux_q8_kcal,vegetaux_q8_kgprot,autres_utilisations/dispo_int,alim_ani/nourriture_huma_ani
produit,,,,,,,,,,,,,,,,,,,,,
Ananas,0.287447,6.350000,131.000000,2049.000000,171.000000,0.000000,0.000000,2013,0.000000,231.000000,...,402.426331,2032.000000,57.000000,3.200510e+08,0.000000,0.000000,8.406686e+11,6.004776e+06,0.000000,0.000000
Bananes,0.737199,10.690000,440.800000,1932.600000,1.400000,0.000000,0.000000,2013,0.000000,195.800000,...,606.704321,1424.600000,68.800000,1.240200e+08,0.000000,0.000000,8.975782e+11,1.095812e+07,0.000000,0.000000
Blé,8.449358,92.566522,2323.500000,3274.391304,9245.217391,-148.891304,2157.934783,2013,331.956522,137.913043,...,2816.553572,6623.586957,352.021739,9.493198e+07,452.869565,130.000000,2.656403e+13,8.215346e+08,3.303929,24.573586
Café,5.916733,5.510000,562.000000,1493.000000,1.500000,30.500000,0.000000,2013,0.000000,155.000000,...,429.707422,963.000000,0.000000,2.013890e+08,0.000000,0.000000,4.128280e+11,5.862252e+07,0.000000,0.000000
"Fruits, Autres",0.583938,28.796250,814.625000,2090.750000,7860.500000,9.750000,0.000000,2013,1.500000,120.250000,...,442.551484,8401.875000,543.375000,2.626550e+08,0.000000,205.000000,3.821409e+12,4.700752e+07,0.016400,0.000000
Huile de Palme,0.452651,3.127500,1907.500000,2647.250000,1700.500000,-58.166667,0.000000,2013,1710.666667,123.583333,...,8715.534676,648.583333,23.500000,3.153898e+08,0.000000,0.250000,7.738228e+12,2.096172e+06,71.811376,0.000000
Lait - Excl Beurre,3.270813,155.952069,2578.655172,3119.448276,12481.275862,56.172414,1051.793103,2013,436.517241,141.000000,...,532.644266,11437.379310,177.413793,1.204236e+08,0.000000,5.344828,NaN,NaN,3.337789,8.421640
"Légumes, Autres",1.304189,85.382727,1263.181818,2561.545455,6618.363636,1.545455,361.818182,2013,0.000000,142.090909,...,283.498412,6882.818182,661.090909,8.852082e+07,2.727273,14.545455,2.078817e+12,1.003994e+08,0.000000,4.994290
Manioc,0.631059,12.408000,5231.400000,6931.000000,11753.000000,12.600000,5029.800000,2013,4071.000000,137.800000,...,1098.781966,3051.400000,951.800000,3.550074e+08,0.000000,360.800000,1.661539e+13,1.150132e+08,30.233491,62.240756


In [43]:
print('Ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure')
print(quinze_produits_importes['autres_utilisations/dispo_int'].sort_values(ascending=False).head(3), '\n')

print('Ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)')
print(quinze_produits_importes['alim_ani/nourriture_huma_ani'].sort_values(ascending=False).head(3))

Ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure
produit
Huile de Palme    71.811376
Manioc            30.233491
Maïs              25.920006
Name: autres_utilisations/dispo_int, dtype: float64 

Ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)
produit
Maïs                   87.822808
Poissons Pelagiques    75.535947
Soja                   64.608860
Name: alim_ani/nourriture_huma_ani, dtype: float64


## Q13 : USA
Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [44]:
# 10% des céréales destinées aux aliments pour animaux, produites par les USA 
usa_cereales_ani = aliments[(aliments['produit'].isin(liste_cereales)) & (aliments['pays']=='États-Unis d\'Amérique')]['alim_ani'].sum() * .1
print(round(usa_cereales_ani), 'milliers de tonnes de céréales')

14010 milliers de tonnes de céréales


In [45]:
# Valeurs nutritionnelles moyennes des céréales
cereales_moyenne_kcal = aliments[aliments['produit'].isin(liste_cereales)]['kcal/kg'].mean()
cereales_moyenne_kgprot = aliments[aliments['produit'].isin(liste_cereales)]['% protéines'].mean()

print('Moyennes des valeurs nutritionnelles des céréales :')
print(round(cereales_moyenne_kcal), 'kcal')
print(round(cereales_moyenne_kgprot), '% \n')

# Personnes qu'on pourrait nourrir avec les 10% de céréales
bouches_cereales_usa_kcal = usa_cereales_ani * 1000000 * cereales_moyenne_kcal / besoin_kcal
bouches_cereales_usa_kgprot = usa_cereales_ani * 1000000 * cereales_moyenne_kgprot /100 / besoin_kgprot

print(
    'Avec les 10% de céréales destinées aux aliments pour animaux des USA, on pourrait potentiellement nourrir : \n\n',
    'En calories :', round(bouches_cereales_usa_kcal / 10**6), 'millions de personnes \n'
    'En protéines :', round(bouches_cereales_usa_kgprot / 10**6), 'millions de personnes \n',
    'Soit en moyenne', round(np.mean([bouches_cereales_usa_kcal,bouches_cereales_usa_kgprot]) / sousnutrition_monde * 100),
    '% de la population mondiale sous-alimentée'
    )



Moyennes des valeurs nutritionnelles des céréales :
2957 kcal
8 % 

Avec les 10% de céréales destinées aux aliments pour animaux des USA, on pourrait potentiellement nourrir : 

 En calories : 49 millions de personnes 
En protéines : 52 millions de personnes 
 Soit en moyenne 7 % de la population mondiale sous-alimentée


## Q14 : Thaïlande
En Thaïlande, quelle proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?

In [46]:
# Exportations de manioc en Thaïlande (milliers de tonnes)
manioc_exports_thai = aliments[(aliments['produit']=='Manioc') & (aliments['pays']=='Thaïlande')]['Exportations - Quantité']

# Proportion des exportations de manioc / le total des exportations
ratio_manioc_exports_thai = manioc_exports_thai / aliments[aliments['pays']=='Thaïlande']['Exportations - Quantité'].sum() * 100

# Ratio de personnes sous-alimentées en Thaïlande
ratio_sousnutrition_thai = sous_nutrition[sous_nutrition['pays']=='Thaïlande']['% sous_nutrition']


print(
    'En Thaïlande : \n\n',
    int(manioc_exports_thai), 'milliers de tonnes de manioc exportées \n',
    '(', int(ratio_manioc_exports_thai), '% du total des exportations ) \n\n',
    round(float(ratio_sousnutrition_thai), 2), '% de la population en sous-nutrition'
)

En Thaïlande : 

 25214 milliers de tonnes de manioc exportées 
 ( 49 % du total des exportations ) 

 8.36 % de la population en sous-nutrition


In [47]:
# Valeurs nutritionnelles du manioc en Thaïlande
manioc_thai_kcal = aliments[(aliments['produit']=='Manioc') & (aliments['pays']=='Thaïlande')]['kcal/kg']
manioc_thai_kgprot = aliments[(aliments['produit']=='Manioc') & (aliments['pays']=='Thaïlande')]['% protéines']

bouches_manioc_thai_kcal = int(manioc_exports_thai * 1000000 * manioc_thai_kcal / besoin_kcal)
bouches_manioc_thai_kgprot = int(manioc_exports_thai * 1000000 * manioc_thai_kgprot / 100 / besoin_kgprot)

population_thai = int(population[population['pays']=='Thaïlande']['population'])

print(
    'Valeurs nutritionnelles du manioc en Thaïlande : \n',
    round(int(manioc_thai_kcal)), 'kcal \n',
    round(float(manioc_thai_kgprot), 2), '% de protéines \n\n',
    'Avec les exportations de manioc, on pourrait nourrir : \n'
    'En calories : ', round(bouches_manioc_thai_kcal / 10**6), 'millions de personnes',
    '(', round(bouches_manioc_thai_kcal / population_thai * 100), '% de la population) \n',
    'En protéines : ', round(bouches_manioc_thai_kgprot / 10**6), 'millions de personnes',
    '(', round(bouches_manioc_thai_kgprot / population_thai * 100), '% de la population) \n'
    )

Valeurs nutritionnelles du manioc en Thaïlande : 
 1123 kcal 
 0.39 % de protéines 

 Avec les exportations de manioc, on pourrait nourrir : 
En calories :  34 millions de personnes ( 50 % de la population) 
 En protéines :  5 millions de personnes ( 7 % de la population) 



# **Questions 15 à 20**

## Q15 : Table population
- Clés candidates : 
    - 'code_pays'
    - 'pays'
- Clé primaire retenue : 'pays'

In [48]:
# Tests pour la clé primaire
print(
    population[population['pays']=='Albanie'], '\n\n',
    population[population['pays']=='Guatemala'],
)

# Export CSV
population.to_csv('exports/population.csv', index=False)

   code_pays     pays  population  annee
2          3  Albanie     3173000   2013 

     code_pays       pays  population  annee
67         89  Guatemala    15468000   2013


## Q16 : Table dispo_alim
- Clés candidates :
    - 'pays' + 'produit'
    - 'pays' + 'code_produit'
    - 'code_pays' + 'produit'
    - 'code_pays' + 'code_produit'
- Clé primaire retenue : 'pays' + 'produit'

In [49]:
# Création de la colonne dispo_alim_tonnes (?)
aliments['dispo_alim_tonnes'] = aliments['Disponibilité alimentaire en quantité (kg/personne/an)'] * aliments['population'] * 365 / 1000

dispo_alim = aliments[[
    'pays',
    'code_pays',
    'annee',
    'produit',
    'code_produit',
    'origine',
    'dispo_alim_tonnes', # ?
    'dispo_alim_kcal_p_j',
    'dispo_prot',
    'dispo_mat_gr'
]]

# Tests pour la clé primaire
print(
    dispo_alim[(dispo_alim['pays']=='Soudan') & (dispo_alim['produit']=='Blé')], '\n\n\n',
    dispo_alim[(dispo_alim['pays']=='Japon') & (dispo_alim['produit']=='Café')],
)

# Export CSV
dispo_alim.to_csv('exports/dispo_alim.csv', index=False)

         pays  code_pays  annee produit  code_produit   origine  \
15531  Soudan        276   2013     Blé          2511  Végétale   

       dispo_alim_tonnes  dispo_alim_kcal_p_j  dispo_prot  dispo_mat_gr  
15531        312056487.2                197.0        6.18          1.18   


        pays  code_pays  annee produit  code_produit   origine  \
6978  Japon        110   2013    Café          2630  Végétale   

      dispo_alim_tonnes  dispo_alim_kcal_p_j  dispo_prot  dispo_mat_gr  
6978        180525408.4                  5.0        0.65           0.0  


## Q17 : Table equilibre_prod
- Clés candidates :
    - 'pays' + 'produit'
    - 'pays' + 'code_produit'
    - 'code_pays' + 'produit'
    - 'code_pays' + 'code_produit'
- Clé primaire retenue : 'pays' + 'produit'

In [50]:
equilibre_prod = aliments[[
    'pays',
    'code_pays',
    'annee',
    'produit',
    'code_produit',
    'dispo_int',
    'alim_ani',
    'semences',
    'pertes',
    'transfo', # ?
    'nourriture',
    'autres_utilisations'
]]

# Tests pour la clé primaire
print(
    equilibre_prod[(equilibre_prod['pays']=='Islande') & (equilibre_prod['produit']=='Vin')],'\n\n\n',
    equilibre_prod[(equilibre_prod['pays']=='Italie') & (equilibre_prod['produit']=='Seigle')],
)

# Export CSV
equilibre_prod.to_csv('exports/equilibre_prod.csv', index=False)

         pays  code_pays  annee produit  code_produit  dispo_int  alim_ani  \
6029  Islande         99   2013     Vin          2655        4.0       0.0   

      semences  pertes  transfo  nourriture  autres_utilisations  
6029       0.0     0.0      0.0         4.0                  0.0   


         pays  code_pays  annee produit  code_produit  dispo_int  alim_ani  \
6667  Italie        106   2013  Seigle          2515       39.0      32.0   

      semences  pertes  transfo  nourriture  autres_utilisations  
6667       1.0     1.0      0.0         6.0                  0.0  


## Q18 : Table sous_nutrition
- Clés candidates : 
    - 'code_pays'
    - 'pays'
- Clé primaire retenue : 'pays'

In [51]:
# Projection
sous_nutrition = sous_nutrition[['pays','code_pays','annee','nb_personnes']]

# Tests pour la clé primaire
print(
    sous_nutrition[sous_nutrition['pays']=='Bulgarie'], '\n\n',
    sous_nutrition[sous_nutrition['pays']=='Colombie'],
)

# Export CSV
sous_nutrition.to_csv('exports/sous_nutrition.csv', index=False)

        pays  code_pays annee  nb_personnes
26  Bulgarie         27  2013      400000.0 

         pays  code_pays annee  nb_personnes
38  Colombie         44  2013     4100000.0


## Q19 : Requêtes SQL

### Pays aux plus hauts ratios
Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, puis en termes de kcal par habitant.

- dispo_prot est exprimé en grammes/habitant/jour
- On divise par 1000 pour obtenir la valeur en kg

```sql
SELECT pays, SUM(dispo_prot/1000) AS sum_prot_kg
FROM dispo_alim
GROUP BY pays
ORDER BY sum_prot_kg DESC
```

En première position, l'Islande obtient 0,133kg de protéines par habitant et par jour, soit largement le double du besoin journalier moyen (environ 55g/habitant/jour)

![title](exports/sum_prot_kg.png)

- dispo_alim_kcal_p_j est déjà exprimé en kcal/habitant/jour

```sql
SELECT pays, SUM(dispo_alim_kcal_p_j) AS sum_kcal
FROM dispo_alim
GROUP BY pays
ORDER BY sum_kcal DESC
```

Les dix premiers pays de cette liste dépassent confortablement le besoin calorique journalier par habitant (environ 2300kcal)

![title](exports/sum_kcal.png)

### Pays aux plus faibles ratios
Pour l'année 2013, les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant.

```sql
SELECT pays, SUM(dispo_prot/1000) AS sum_prot_kg
FROM dispo_alim
GROUP BY pays
ORDER BY sum_prot_kg ASC
```

Avec des disponibilités allant de 37 à 55 grammes, ces dix pays se situent tous au dessous du minimum de 55 grammes de protéines par habitant et par jour. On franchit le seuil de ces 55 grammes dès le 12ème pays de cette liste (Zambie). 

![title](exports/sum_prot_kg_asc.png)

### Volumes des pertes
La quantité totale (en kg) de produits perdus par pays en 2013.

- pertes est exprimé en milliers de tonnes
- On multiplie par 1000000 pour obtenir la valeur en kg

```sql
SELECT pays, SUM(pertes * 1000000) AS sum_pertes
FROM equilibre_prod
GROUP BY pays
```

![title](exports/sum_pertes.png)

Si on classe les pays par volume de produits perdus, on réalise que les dix premiers pays de cette liste appartiennent pour moitié à la liste des pays sous-alimentés.
```sql
SELECT pays, SUM(pertes * 1000000) AS sum_pertes
FROM equilibre_prod
GROUP BY pays
ORDER BY sum_pertes DESC
```

In [52]:
pays_pertes_desc = pd.Series([
    'Chine, continentale',
    'Brésil',
    'Inde',
    'Nigéria',
    'Indonésie',
    'Turquie',
    'Mexique',
    'Égypte',
    'Ghana',
    'États-Unis d\'Amérique'
])

pd.DataFrame([
    pays_pertes_desc,
    pays_pertes_desc.isin(liste_pays_sousnutrition)
])

,0,1,2,3,4,5,6,7,8,9
0,"Chine, continentale",Brésil,Inde,Nigéria,Indonésie,Turquie,Mexique,Égypte,Ghana,États-Unis d'Amérique
1,True,False,True,True,True,False,False,False,True,False


A l'inverse, aucun des 10 pays accusant le moins de pertes ne figure dans la liste des pays sous-alimentés.
```sql
SELECT pays, SUM(pertes * 1000000) AS sum_pertes
FROM equilibre_prod
GROUP BY pays
ORDER BY sum_pertes ASC
```

In [53]:
pays_pertes_asc = pd.Series([
    'Antigua-et-Barbuda',
    'Bermudes',
    'Islande',
    'Kiribati',
    'Maldives',
    'Saint-Kitts-et-Nevis',
    'Chine - RAS de Macao',
    'Bahamas',
    'Barbade',
    'Grenade'
])

pd.DataFrame([
    pays_pertes_asc,
    pays_pertes_asc.isin(liste_pays_sousnutrition)
])

,0,1,2,3,4,5,6,7,8,9
0,Antigua-et-Barbuda,Bermudes,Islande,Kiribati,Maldives,Saint-Kitts-et-Nevis,Chine - RAS de Macao,Bahamas,Barbade,Grenade
1,False,False,False,False,False,False,False,False,False,False


### Proportion de la sous-nutrition
Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.

On joint d'abord population et sous_nutrition dans une nouvelle table (2 méthodes possibles) :
```sql    
CREATE TEMP TABLE sous_nutrition_ratio AS
SELECT * FROM population, sous_nutrition
WHERE population.pays = sous_nutrition.pays
```

Puis :
```sql    
SELECT pays, ( nb_personnes / population ) AS ratio
FROM sous_nutrition_ratio
ORDER BY ratio DESC
```

6 de ces pays apparaissaient déjà dans la liste des pays au plus faible ratio disponibilité alimentaire/habitant en termes de protéines par habitant (Liberia, République centrafricaine, Madagascar, Haïti, Zimbabwe et Congo).

![title](exports/pays_sousnutrition.png)

[Analyse plus appronfondie sur ce résultat ↓](#Analyse-des-10-pays-les-plus-sous-alimentés)

### Produits au plus haut ratio autres/dispo_int
Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.

```sql    
SELECT produit, ( autres_utilisations / dispo_int ) AS ratio
FROM equilibre_prod
GROUP BY produit
ORDER BY ratio DESC
```

Seuls 6 produits ont un ratio (autres utilisations / dispo. intérieure) supérieur à 0. Tous sont des huiles et graisses, ou des produits dont on extrait l'huile.

![title](exports/autres_dispo.png)

## Q20 : Autres utilisations des produits
Pour quelques uns des produits identifiés dans cette dernière requête SQL, supposez quelles sont ces "autres utilisations" possibles (recherchez sur internet !)

La question précédente a montré que  :
    
       - Huile de palme
       - Arachides décortiquées
       - Huile de plantes oléifères
       - Sorgho
       - Plantes oléifères
       - Graisses animales crues

Les huiles et graisses de ces produits sont très utilisés dans l'alimentation industrielle, les cosmétiques et carburants. 

**Huile de palme** : majoritairement utilisée dans l'élaboration de plats préparés, gâteaux et biscuits, mais aussi pour l'oléochimie (cosmétique, peintures et lubrifiants).

**Arachides décortiquées** : on en extrait de l'huile tranformée en margarine ou utilisée comme huile de table. Cette huile est également utilisée dans la savonnerie et la pharmacopée.

**Huile de plante oléifère** et **plantes oléifères** (colza, tournesol, olivier) : hormis l'utilisation de ces huiles pour l'alimentation humaine, on en produit également des biocarburants.  

**Sorgho** : on en extrait du sucre et des alcools, mais aussi de l'agrocarburant et des biomatériaux.

**Graisses animales crues** : servent de base en savonnerie, cosmétique et même parfumerie

Ils affichent tous une forte valeur calorique ↓

In [54]:
aliments[aliments['produit'].isin(
    [
        'Huile de Palme',
        'Arachides Decortiquees',
        'Huil Plantes Oleif Autr',
        'Sorgho',
        'Plantes Oleiferes, Autre',
        'Graisses Animales Crue'
    ]
)].groupby(['produit']).mean()[['kcal/kg','% protéines']]

,kcal/kg,% protéines
produit,,
Arachides Decortiquees,5702.900100,25.146613
Graisses Animales Crue,7583.862063,1.450564
Huil Plantes Oleif Autr,8848.222893,1.104687
Huile de Palme,8719.419872,0.492657
"Plantes Oleiferes, Autre",4350.089305,17.054422
Sorgho,3060.415548,8.709284


# **Analyses supplémentaires**

## Chiffres

### Décès dus à la faim
25000 personnes meurent de faim chaque jour ([Source : FAO](http://www.fao.org/french/newsroom/news/2002/9703-fr.html))

In [55]:
morts_jour = 25000
morts_an = morts_jour * 365
print(
    'Soit', round(morts_an / 10**6), 'millions de morts par an (',
    round((morts_an / population_monde * 100), 2),'% de la population mondiale )')

Soit 9 millions de morts par an ( 0.13 % de la population mondiale )


### Nombre de personnes sous-alimentées

In [56]:
print(
    'En 2013,', round(sousnutrition_monde / 10**6), 'millions de personnes dans le monde étaient en sous-nutrition \n',
    '(', round(ratio_sousnutrition * 100), '% de la population mondiale )'
)

En 2013, 744 millions de personnes dans le monde étaient en sous-nutrition 
 ( 11 % de la population mondiale )


### Évolutions possibles pour 2050
Les Nations Unies prévoient que la population mondiale devrait avoisinner 9,7 milliards de personnes en 2050 ([Source : un.org](https://www.un.org/fr/sections/issues-depth/population/index.html#:~:text=Selon%20les%20projections%2C%20la%20population,individus%20vers%20l'an%202100.))

In [57]:
population_monde_2050 = 9.7 * 10**9
ratio_2050_2013 = population_monde_2050 / population_monde
print(
    'Soit', round(ratio_2050_2013 * 100), '% de la population mondiale estimée en 2013.'
)

Soit 139 % de la population mondiale estimée en 2013.


In [58]:
print(
    'Sans évolution positive de la malnutrition, en 2050 :\n', 
    '-', round((population_monde_2050 * ratio_sousnutrition / 10**9), 2),
    'milliard de personnes seront sous-alimentées \n'

    ' -', round(morts_an * ratio_2050_2013 / 10**6),
    'millions de personnes mourront de la faim'

)

Sans évolution positive de la malnutrition, en 2050 :
 - 1.03 milliard de personnes seront sous-alimentées 
 - 13 millions de personnes mourront de la faim


## Causes de la sous-alimentation
Parmi les causes communément admises, on peut citer :
   - Les changements et catastrophes climatiques
   - La problématique de l'eau
   - Les guerres et conflits
   - Les inégalités et la pauvreté
   - La croissance démographique

### Table des 10 pays les plus sous-alimentés
La table ci-dessous détaille la relation des [10 pays les plus sous-alimentés au monde](#Proportion-de-la-sous-nutrition) à chacune de ces causes ↓

In [59]:
sous_nutrition_10pays = pd.read_csv('exports/sous_nutrition_10pays.csv')
sous_nutrition_10pays

,pays,catastrophes et changement climatique,accès à l'eau,pauvreté,croissance démographique (2013),conflit (territorial),fin conflit,occupation,libération,régime
0,Haïti,"tempêtes, séismes",faible,"78,00%","1,30%",renversement,1995,étrangère,1934,République
1,Zambie,sécheresse,faible,"87,00%","3,10%",guerre,1988,coloniale,1964,République
2,Zimbabwe,sécheresse,moyen,"74,00%","1,80%",guerre civile,1979,coloniale,1964,République
3,République centrafricaine,inondations,faible,"92,00%","0,30%",guerre civile,2014,coloniale,1960,République
4,République populaire démocratique de Corée,inondations,moyen,?,"0,50%",guerre,actif,étrangère,1948,République
5,Congo,inondations,faible,"82,00%","3,30%",guerre civile,2003,coloniale,1960,République
6,Tchad,"sécheresse, inondations",faible,"86,00%","3,40%",guerre civile,2010,coloniale,1960,République
7,Angola,inondations,faible,"87,00%","3,60%",guerre civile,2002,coloniale,1975,République
8,Libéria,montée des eaux,faible,"92,00%","2,70%",guerre civile,2003,-,1848,République
9,Madagascar,montée des eaux,faible,"97,00%","2,70%",guerre,1945,coloniale,1960,République


On remarque des similitudes quasi-systématiques entre ces 10 pays :

- de très forts taux de pauvreté (ressources inférieures à 5,50 dollars par personne et par jour)
- une croissance démographique forte
- un accès à l'eau potable encore restreint
- une récurrence des épisodes de sécheresse et d'inondations
- un conflit armé ces 30 dernières année
- une domination étrangère jusqu'aux années 60


A titre de comparaison, 3 pays n'appartenant pas à cette liste figurent en queue de liste.


Source :
[Wikipedia : percentage_of_population_living_in_poverty ](https://en.wikipedia.org/wiki/List_of_countries_by_percentage_of_population_living_in_poverty)